In [1]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql.functions import col, udf, lag, rank, lit
from pyspark.sql.window import Window
import talib

In [2]:
global DoEval #是否進行模型評估

next_date_range_12 = ["20180619", "20180620", "20180621", "20180622"] #設定預測區間
next_date_range_11 = ["20180611", "20180612", "20180613", "20180614", "20180615"] #設定預測區間
next_date_range_10 = ["20180604", "20180605", "20180606", "20180607", "20180608"] #設定預測區間
next_date_range_9 = ["20180528", "20180529", "20180530", "20180531", "20180601"] #設定預測區間
next_date_range_8 = ["20180521", "20180522", "20180523", "20180524", "20180525"] #設定預測區間
next_date_range_7 = ["20180514", "20180515", "20180516", "20180517", "20180518"] #設定預測區間
ignore_dates = [""]#設定排除日(如端午節)
next_date_range_6 = ["20180507", "20180508", "20180509", "20180510", "20180511"] #設定預測區間

train_date_range = [next_date_range_6, next_date_range_7, next_date_range_8, next_date_range_9, next_date_range_10, next_date_range_11 ]
pred_date_range = next_date_range_12

predict_start_date = pred_date_range[0]

if sc.master[0:5]=="local":
#     Path = "file:/c:/D Drive/work/bigData/pySpark/TBrain_Round2_DataSet_20180601"
    #Path = "file:/Users/yungchuanlee/Documents/learn/AI競賽/ETF預測/TBrain_Round2_DataSet_20180511"
    Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180615"
else:
    Path = "hdfs://master:9000/user/hduser"
Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180615"

In [3]:
#define alias of columns
col_alias_etf= {"代碼":"etf_id", "日期": "etf_date", "中文簡稱": "etf_name", "開盤價(元)":"etf_open", 
            "最高價(元)":"etf_high", "最低價(元)":"etf_low", "收盤價(元)":"etf_close", "成交張數(張)":"etf_count"}
col_alias_stock= {"代碼":"stock_id", "日期": "stock_date", "中文簡稱": "stock_name", "開盤價(元)":"stock_open", 
            "最高價(元)":"stock_high", "最低價(元)":"stock_low", "收盤價(元)":"stock_close", "成交張數(張)":"stock_count"}#udf
def to_double(str_val):
    return float(str_val.replace(",",""))
to_double=udf(to_double)

In [4]:
#def function to read data (因檔案格式都相同)
def read_data(file_name, col_alias):
    str_cols = ["代碼","日期", "中文簡稱"]
    raw_data = spark.read.option("encoding", "Big5").csv(Path + "/" + file_name, header=True, sep=",")
    print("Total " + file_name + " count: " + str(raw_data.count()))
    #rename cols and correct type 
    num_cols = [col_name for col_name in raw_data.columns if col_name not in str_cols]
    final_data=raw_data.select( [col(str_col_name).alias(col_alias[str_col_name]) for str_col_name in str_cols] + 
                                  [to_double(col(num_col_name)).cast("double").alias(col_alias[num_col_name]) for num_col_name in num_cols] )
    final_data.printSchema()
    final_data.show(5)
    return final_data

In [5]:
print("starting import tetfp.csv(台灣18檔ETF股價資料)...")
tetfp_dt=read_data("tetfp.csv", col_alias_etf)

starting import tetfp.csv(台灣18檔ETF股價資料)...
Total tetfp.csv count: 19665
root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)

+-------+--------+----------------+--------+--------+-------+---------+---------+
| etf_id|etf_date|        etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|
+-------+--------+----------------+--------+--------+-------+---------+---------+
|0050   |20130102|元大台灣50          |    54.0|   54.65|   53.9|     54.4|  16487.0|
|0050   |20130103|元大台灣50          |    54.9|   55.05|  54.65|    54.85|  29020.0|
|0050   |20130104|元大台灣50          |   54.85|   54.85|   54.4|     54.5|   9837.0|
|0050   |20130107|元大台灣50          |   54.55|   54.55|   53.9|    54.25|   8910.0|
|0050   |20130108|元大台灣50     

In [6]:
#取出etf的distinct id
etf_ids = []
etf_idx_dic = {}
etf_idx = 0
for row in tetfp_dt.select("etf_id").distinct().orderBy("etf_id").collect():
    etf_ids.append(row["etf_id"])
    etf_idx_dic.update({row["etf_id"]: etf_idx})
    etf_idx += 1
etf_idx_dic
# etf_ids = ['0050   ']

{'0050   ': 0,
 '0051   ': 1,
 '0052   ': 2,
 '0053   ': 3,
 '0054   ': 4,
 '0055   ': 5,
 '0056   ': 6,
 '0057   ': 7,
 '0058   ': 8,
 '0059   ': 9,
 '006201 ': 10,
 '006203 ': 11,
 '006204 ': 12,
 '006208 ': 13,
 '00690  ': 14,
 '00692  ': 15,
 '00701  ': 16,
 '00713  ': 17}

In [7]:
etf_dic = {}
for etfid in etf_ids:
    export_dt = tetfp_dt.filter("etf_id='" +etfid+ "' and etf_date < '" + predict_start_date + "'") \
        .orderBy("etf_id", "etf_date", ascending=True)
    export_pd = export_dt.toPandas()
    etf_dic.update({etfid.strip(): export_pd})

In [8]:
#functions of TA lib
#print(talib.get_functions())
print(tetfp_dt.filter("etf_date < '20180611'").count())

19575


In [9]:
# etf_dic["0050"]["etf_close"].count()

In [10]:
import math
def avg_list(p_list):
    #計算數字list的平均值
    return sum(p_list)/len(p_list)

def get_feature_pre(curr_list):
    #在feature前加上一個Nan後回傳, 第二個值則回傳原本的最後一個值作為下次的feature
    if type(curr_list) is pd.Series:
        rtn_list = curr_list.tolist()
    else:
        rtn_list = curr_list
    rtn_list = [None] + rtn_list
    return (rtn_list[:-1], [rtn_list[-1]])

def checkNan(num):
    if num == None:
        return True
    elif math.isnan(num):
        return True
    else:
        return False
    
def calculate_n_bias(close_price, ma):
#     (close price - MA)/MA
    if checkNan(close_price) or checkNan(ma):
        return None
    else:
        return (close_price - ma)/ma
    
def calculate_n_bias_list(close_ser, ma_list):
    #calculate n bias (20 days)
    bias_list = []
    for cprice, ma in zip(close_ser.tolist(), ma_list):
        bias_list.append(calculate_n_bias(cprice, ma))
    return bias_list

def calculate_bias_ma_list(ma3_list, ma6_list):
    #calculate bias_ma = (ma3-ma6)/ma6
    bias_list = []
    for ma3, ma6 in zip(ma3_list, ma6_list):
        bias_list.append(calculate_n_bias(ma3, ma6))
    return bias_list

def calculate_bias_3_6_list(bias3_list, bias6_list):
    #bias_3_6 = bias3 - bias6
    bias_list = []
    for bias3, bias6 in zip(bias3_list, bias6_list):
        if checkNan(bias3) or checkNan(bias6):
            bias_list.append(None)
        else:
            bias_list.append(bias3 - bias6)
    return bias_list

def calculate_dif(ema12_list, ema26_list):
    #差離值DIF = 12日EMA - 26日EMA 
    dif_list = []
    for ema12, ema26 in zip(ema12_list, ema26_list):
        if checkNan(ema12) or checkNan(ema26):
            dif_list.append(None)
        else:
            dif_list.append(ema12 - ema26)
    return dif_list
def calculate_slope(cur_list, prev_list):
    slope_list = []
    for cur, prev in zip(cur_list, prev_list):
        if checkNan(cur) or checkNan(prev):
            slope_list.append(None)
        else:
            slope_list.append((cur-prev)/prev)
    return slope_list

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import itertools
def create_feature(etf_pd, next_date, scaler_dic):
    close_ser = etf_pd["etf_close"]
    #EMA
    ema3 = talib.EMA(close_ser,timeperiod=3)
    ema3_prev, _ = get_feature_pre(ema3)
    ema3_slope = calculate_slope(ema3, ema3_prev)
    ema5 = talib.EMA(close_ser,timeperiod=5)
    ema5_prev, _ = get_feature_pre(ema5)
    ema5_slope = calculate_slope(ema5, ema5_prev)
    ema10 = talib.EMA(close_ser,timeperiod=10)
    ema10_prev, _ = get_feature_pre(ema10)
    ema10_slope = calculate_slope(ema10, ema10_prev)
    ema20 = talib.EMA(close_ser,timeperiod=20)
    ema30 = talib.EMA(close_ser,timeperiod=30)
    
    #make diff
    close_ser_prev, _ = get_feature_pre(close_ser)
    close_diff = calculate_dif(close_ser.tolist(), close_ser_prev)
    
    close_diff_2, _ = get_feature_pre(close_diff)
    close_diff_3, _ = get_feature_pre(close_diff_2)
    #BIAS
    #nBIAS -3, 6, 20 => (close price - MA)/MA   ,Paper 建議用20日MA
    ma3 = talib.MA(np.array(close_ser), timeperiod=3)
    ma6 = talib.MA(np.array(close_ser), timeperiod=6)
    ma20 = talib.MA(np.array(close_ser), timeperiod=20)
    nbias3 = calculate_n_bias_list(close_ser, ma3)
    nbias6 = calculate_n_bias_list(close_ser, ma6)
    nbias20 = calculate_n_bias_list(close_ser, ma20)
    
    #BIAS_ma = (ma3-ma6)/ma6
    bias_ma = calculate_bias_ma_list(ma3, ma6)
    
    #bias_3_6 = bias3 - bias6
    bias_3_6 = calculate_bias_3_6_list(nbias3, nbias6)
    
    #KD --> only use STOCHRSI
    k, d = talib.STOCHRSI(close_ser, timeperiod=9, fastk_period=3, fastd_period=3, fastd_matype=0)
    
    #差離值DIF = 12日EMA - 26日EMA 
    ema12 = talib.EMA(close_ser,timeperiod=12)
    ema26 = talib.EMA(close_ser,timeperiod=26)
    dif = calculate_dif(ema12, ema26)
    
    #MACD
#     macd1, macdsignal1, macdhist1 = talib.MACD(close_ser, fastperiod=12, slowperiod=26, signalperiod=9)
    macd1, macdsignal1, macdhist1 = talib.MACDEXT(close_ser, fastperiod=12, fastmatype=0, 
                                               slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
#     macd1, macdsignal1, macdhist1 = talib.MACDFIX(close_ser, signalperiod=9)
    #RSI
    rsi = talib.RSI(close_ser, timeperiod=10)
    
    #CMO
    cmo = talib.CMO(close_ser, timeperiod=14)
    
    #ROCR - Rate of change ratio: (price/prevPrice)
    #rocr = talib.ROCR(close_ser, timeperiod=10)
    #ROC - Rate of change : ((price/prevPrice)-1)*100
    rocr = talib.ROC(close_ser, timeperiod=10)
    
    #PPO - Percentage Price Oscillator
    ppo = talib.PPO(close_ser, fastperiod=12, slowperiod=26, matype=0)
    #APO - Absolute Price Oscillator
    apo = talib.APO(close_ser, fastperiod=12, slowperiod=26, matype=0)
    
    #MOM - Momentum
    mom = talib.MOM(close_ser, timeperiod=10)
    #TRIX - 1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
    trix = talib.TRIX(close_ser, timeperiod=30)
    #作標準化
    etfid = etf_pd["etf_id"].tolist()[0]
    tmp_pd = pd.DataFrame({"etf_id": etfid, "etf_date": etf_pd["etf_date"], "etf_close": close_ser, "close_dif": close_diff,  
#                            "close_diff_1": close_diff, "close_diff_2": close_diff_2, "close_diff_3": close_diff_3,
                           "ema3":ema3, "ema5": ema5, "ema10": ema10, "ema20": ema20, "ema30" : ema30,
                           "ema3_slope": ema3_slope, "ema5_slope": ema5_slope, "ema10_slope": ema10_slope,
                          "nbias3": nbias3, "nbias6": nbias6, "nbias20": nbias20, "bias_ma": bias_ma,
                          "bias_3_6": bias_3_6, "k": k, "d": d, "dif": dif,
                          #"macd": macd, "macdsignal": macdsignal, "macdhist": macdhist,
                          "macd1": macd1, "macdsignal1": macdsignal1, "macdhist1": macdhist1,
                          "rsi": rsi, "cmo": cmo, "rocr": rocr, "ppo": ppo}).dropna(how='any')
    tmp_pd = tmp_pd[tmp_pd['etf_date'] > '20140101']
    #exclude outlier
#     for col in [c for c in tmp_pd.columns.values if c not in ["etf_id", "etf_date"]]:
#         tmp_pd = tmp_pd[np.abs(tmp_pd[col] - tmp_pd[col].mean()) <= 3.0*tmp_pd[col].std()] #只取3倍以內標準差資料
    
    std_dic = {}
    for col in [c for c in tmp_pd.columns.values if c not in ["etf_id", "etf_date"]]:
#         scaler_key = etfid.strip() + '-' + col
        scaler_key = col
        if scaler_key in scaler_dic:
            scaler = scaler_dic[scaler_key]
            merged = list(itertools.chain.from_iterable(scaler.transform(tmp_pd[[col]])))                
#             merged = tmp_pd[col]
        else:
            scaler = MinMaxScaler()
            merged = list(itertools.chain.from_iterable(scaler.fit_transform(tmp_pd[[col]])))
            merged = tmp_pd[col]
            #保留etf_close的scaler在後續使用
            scaler_dic.update({scaler_key: scaler})
        std_dic.update({col: merged})
    
    #作出目前的feature table及預測用的feature table
    ema3, next_ema3 = get_feature_pre(std_dic["ema3"])
    ema3_slope, next_ema3_slope = get_feature_pre(std_dic["ema3_slope"])
    ema5, next_ema5 = get_feature_pre(std_dic["ema5"])
    ema5_slope, next_ema5_slope = get_feature_pre(std_dic["ema5_slope"])
    ema10, next_ema10 = get_feature_pre(std_dic["ema10"])
    ema10_slope, next_ema10_slope = get_feature_pre(std_dic["ema10_slope"])
    ema20, next_ema20 = get_feature_pre(std_dic["ema20"])
    ema30, next_ema30 = get_feature_pre(std_dic["ema30"])
    
    nbias3, next_nbias3 = get_feature_pre(std_dic["nbias3"])
    nbias6, next_nbias6 = get_feature_pre(std_dic["nbias6"])
    nbias20, next_nbias20 = get_feature_pre(std_dic["nbias20"])
    
    bias_ma, next_bias_ma = get_feature_pre(std_dic["bias_ma"])
    bias_3_6, next_bias_3_6 = get_feature_pre(std_dic["bias_3_6"])
    k, next_k = get_feature_pre(std_dic['k'])
    d, next_d = get_feature_pre(std_dic['d'])
    dif, next_dif = get_feature_pre(std_dic['dif'])
    
    #macd, next_macd = get_feature_pre(std_dic['macd'])
    #macdsignal, next_macdsignal = get_feature_pre(std_dic['macdsignal'])
    #macdhist, next_macdhist = get_feature_pre(std_dic['macdhist'])
    macd1, next_macd1 = get_feature_pre(std_dic['macd1'])
    macdsignal1, next_macdsignal1 = get_feature_pre(std_dic['macdsignal1'])
    macdhist1, next_macdhist1 = get_feature_pre(std_dic['macdhist1'])
                                               
    rsi, next_rsi = get_feature_pre(std_dic['rsi'])
    cmo, next_cmo = get_feature_pre(std_dic['cmo'])
    rocr, next_rocr = get_feature_pre(std_dic['rocr'])
    ppo, next_ppo = get_feature_pre(std_dic['ppo'])
    
#     close_diff_1, next_close_diff_1 = get_feature_pre(std_dic['close_diff_1'])
#     close_diff_2, next_close_diff_2 = get_feature_pre(std_dic['close_diff_2'])
#     close_diff_3, next_close_diff_3 = get_feature_pre(std_dic['close_diff_3'])
    
    #"apo": apo, "mom": mom, "trix": trix
#     apo, next_apo = get_feature_pre(std_dic['apo'])
#     mom, next_mom = get_feature_pre(std_dic['mom'])
#     trix, next_trix = get_feature_pre(std_dic['trix'])
#     print(len(std_dic['etf_close']), ' ', len(close_diff_1), ' ', len(close_diff_2), ' ' , len(rsi))
    
    rtn_pd = pd.DataFrame({"etf_id": etfid, "idx": etf_idx_dic[etfid], "etf_date": tmp_pd["etf_date"], 
                           "etf_close": std_dic["etf_close"], "close_dif": std_dic['close_dif'], 
                           'ema3': ema3, "ema5": ema5, "ema10": ema10, "ema20": ema20, "ema30" : ema30,
                          "ema3_slope": ema3_slope, "ema5_slope": ema5_slope, "ema10_slope": ema10_slope,
                           "nbias3": nbias3, "nbias6": nbias6, "nbias20": nbias20, "bias_ma": bias_ma,
                          "bias_3_6": bias_3_6, "k": k, "d": d, "dif": dif,
                          #"macd": macd, "macdsignal": macdsignal, "macdhist": macdhist,
                           "macd1": macd1, "macdsignal1": macdsignal1, "macdhist1": macdhist1,
                          "rsi": rsi, "cmo": cmo, "rocr": rocr, "ppo": ppo}).dropna(how='any')
    #col = 'close_dif'
    #rtn_pd = rtn_pd[np.abs(rtn_pd[col] - rtn_pd[col].mean()) <= 3.0*rtn_pd[col].std()] #只取3倍以內標準差資料
    
    next_pd = pd.DataFrame({"etf_id": etfid, "idx": etf_idx_dic[etfid], "etf_date": next_date, 
                            'etf_close': -1.0, "close_dif": -1.0, 
                           "ema3": next_ema3, "ema5": next_ema5, "ema10": next_ema10, "ema20": next_ema20, "ema30" : next_ema30,
                          "ema3_slope": next_ema3_slope, "ema5_slope": next_ema5_slope, "ema10_slope": next_ema10_slope,
                            "nbias3": next_nbias3, "nbias6": next_nbias6, "nbias20": next_nbias20, "bias_ma": next_bias_ma,
                          "bias_3_6": next_bias_3_6, "k": next_k, "d": next_d, "dif": next_dif,
                          #"macd": next_macd, "macdsignal": next_macdsignal, "macdhist": next_macdhist,
                            "macd1": next_macd1, "macdsignal1": next_macdsignal1, "macdhist1": next_macdhist1,
                          "rsi": next_rsi, "cmo": next_cmo, "rocr": next_rocr, "ppo": next_ppo})
    
    return (rtn_pd, next_pd, scaler_dic)

In [11]:
# val = etf_dic["0050"][["etf_close"]]
# # print(val["etf_close"].tolist())
# from sklearn.preprocessing import StandardScaler
# import itertools
# scaler = StandardScaler()
# scaler.fit_transform(val)
# #merged = list(itertools.chain.from_iterable(scaler.fit_transform(val)))
# #merged
# scaler.inverse_transform([-1.44880072])[0]

In [12]:
#create feature pandas
etf_pd_dic = {}
next_pd_dic = {}
scaler_dic = {}
for etfid in etf_ids:
    etf_pd, next_pd, scalers = create_feature(etf_dic[etfid.strip()], predict_start_date, {})
    etf_pd_dic.update({etfid.strip() : etf_pd})
    next_pd_dic.update({etfid.strip() : next_pd})
    scaler_dic.update(scalers)

In [13]:
# etf_pd_dic["0050"]['etf_close'].describe()

In [14]:
#etf_pd_dic["0054"].describe()

In [15]:
#etf_pd_dic["00692"].describe()

In [16]:
from sklearn.preprocessing import MinMaxScaler
import itertools
# pd_0050=etf_pd_dic["0051"]
# pd_0050=pd_0050[pd_0050.etf_date > '20180101']
# scaler = MinMaxScaler()
# pd_0050['close_dif']=list(itertools.chain.from_iterable(scaler.fit_transform(pd_0050[['close_dif']])))
# # pd_0050[['close_dif']].plot()
# for col in list(pd_0050.columns.values): #["bias_3_6", "ema5_slope", 'ema10_slope']:
#     pd_0050[col]=list(itertools.chain.from_iterable(scaler.fit_transform(pd_0050[[col]])))
#     pd_0050[['close_dif',col]].plot()
# plt.show()

In [17]:
#non_feature_list = ['etf_id', 'etf_date', 'close_dif', 'etf_close', 'idx']
#feature_cols = [col for col in list(etf_pd_dic["0050"].columns.values) if col not in non_feature_list]
# feature_cols = ['ema5']
#print(feature_cols)

In [18]:
list('bias_3_6,ema20,ema30,macd1'.split(','))

['bias_3_6', 'ema20', 'ema30', 'macd1']

In [19]:
import xgboost as xgb
def trainModel(train_dt, feature_cols, f_type):
    train_x1 = train_dt[feature_cols].values
    if f_type == '1':
        #model for close_dif
        train_y1 = train_dt["close_dif"].values
        #$$$ parameter
        model1 = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75, 
            colsample_bytree=1, max_depth=68, n_jobs=4, min_child_weight=0 )
        model1.fit(train_x1, train_y1)
        return model1
    else:
        #model for etf_close
        train_y2 = train_dt["etf_close"].values
        model2 = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75, 
            colsample_bytree=1, max_depth=68, n_jobs=4, min_child_weight=0 )
        model2.fit(train_x1, train_y2)
        return model2

In [20]:
eval_date_range = []
for date_range in train_date_range:
    eval_date_range += date_range
eval_date_range += pred_date_range

pd_0050=etf_dic["0050"]
prev_date = pd_0050.loc[~pd_0050["etf_date"].isin(eval_date_range)]["etf_date"].max()
prev_date

'20180504'

In [21]:
from pyspark.sql.types import ArrayType, DoubleType, IntegerType
#計算上或下的值(udf)
def judge_up_down_pred_native(curr_price, prev_close_price):
    if checkNan(prev_close_price):
        return 0.0
    else:
        prev_price = prev_close_price
        if curr_price == prev_price:
            return 0.0
        elif curr_price > prev_price:
            return 1.0
        else:
            return 2.0
judge_up_down_pred=udf(judge_up_down_pred_native, DoubleType())

In [22]:
#declare previous row windows
wsSpec_etf = Window.partitionBy('etf_id').orderBy('etf_date') #time window for normal case
#eval dt for evluate score
tetf_dt_eval = tetfp_dt.filter(tetfp_dt.etf_date.isin([prev_date] + eval_date_range)) \
    .withColumn("prev_close", lag(tetfp_dt.etf_close).over(wsSpec_etf)) 
tetf_dt_eval =  tetf_dt_eval.withColumn("act_ud", judge_up_down_pred(tetf_dt_eval.etf_close, tetf_dt_eval.prev_close))
# tetf_dt_eval = tetf_dt_eval.filter(tetf_dt_eval.etf_date.isin(eval_date_range))
tetf_dt_eval.cache()
tetf_dt_eval.filter("etf_date = '20180507'").show()

+-------+--------+-----------------+--------+--------+-------+---------+---------+----------+------+
| etf_id|etf_date|         etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|prev_close|act_ud|
+-------+--------+-----------------+--------+--------+-------+---------+---------+----------+------+
|00701  |20180507|    國泰臺灣低波動30    |    20.9|    20.9|  20.71|    20.75|     36.0|     20.75|   0.0|
|0051   |20180507| 元大中型100         |   31.96|   32.03|  31.88|     32.0|     16.0|      31.8|   1.0|
|0057   |20180507| 富邦摩台            |    48.7|    48.7|  48.31|    48.63|     10.0|     48.46|   1.0|
|006203 |20180507| 元大MSCI台灣        |   36.98|   36.98|  36.94|    36.94|      3.0|      36.7|   1.0|
|0052   |20180507| 富邦科技            |   51.25|   51.25|   50.9|     51.0|     20.0|     51.15|   2.0|
|0050   |20180507| 元大台灣50          |    79.5|   79.55|   78.8|     79.5|   3858.0|     78.95|   1.0|
|0055   |20180507| 元大MSCI金融        |   17.18|   17.18|  17.03|     17.1|    113.0|     17.0

In [23]:
tetf_pd_eval = tetf_dt_eval.toPandas()
# tetf_pd_eval.loc[(tetf_pd_eval['etf_id'] == '006203 ') & (tetf_pd_eval['etf_date'] == '20180522')]['etf_close'].values[0]

In [24]:
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score
def doEvaluate(predict_res_final, tetf_pd_eval, next_date_range):
    etf_close_list = []
    act_ud_list = []
    prediction_list = []
    pred_ud_list = []
    acc_list = []
    weights = [0.1, 0.15, 0.2, 0.25, 0.3]
    judge_score = 0.0
    for row in predict_res_final:
        eid = row['etf_id']
        edate = row['etf_date']
        if edate not in next_date_range:
            continue
        etf_eval = tetf_pd_eval.loc[(tetf_pd_eval['etf_id'] == eid) & (tetf_pd_eval['etf_date'] == edate)]
        etf_close = etf_eval['etf_close'].values[0]
        act_ud = etf_eval['act_ud'].values[0]
        prediction = row['prediction']
        pred_ud = row['pred_ud']
        #etf score
        eidx = next_date_range.index(edate)
        judge_score = judge_score + \
            ((0.5 if pred_ud == act_ud else 0.0) + \
             ((etf_close-abs(prediction-etf_close))/etf_close)*0.5)*weights[eidx]
        etf_close_list.append(etf_close)
        act_ud_list.append(act_ud)
        prediction_list.append(prediction)
        pred_ud_list.append(pred_ud)
        acc_list.append(1.0 if pred_ud == act_ud else 0.0)
    #evaluate accuracy_score
    accuracy = accuracy_score(act_ud_list, pred_ud_list, normalize=True)
#     print(act_ud_list)
#     print(pred_ud_list)
    rmse = mean_squared_error(etf_close_list, prediction_list)
    return (rmse, accuracy, judge_score)
    

In [25]:
scaler_dic.keys()

dict_keys(['bias_3_6', 'bias_ma', 'close_dif', 'cmo', 'd', 'dif', 'ema10', 'ema10_slope', 'ema20', 'ema3', 'ema30', 'ema3_slope', 'ema5', 'ema5_slope', 'etf_close', 'k', 'macd1', 'macdhist1', 'macdsignal1', 'nbias20', 'nbias3', 'nbias6', 'ppo', 'rocr', 'rsi'])

In [26]:
def max_7percent(prev_close, prediction):
    dif = prediction - prev_close
    pct = 0.07
    dif_max = prev_close * pct
    rtn_val = prediction
    if abs(dif) > dif_max:
        if dif > 0:
            rtn_val = prev_close + dif_max
        else:
            rtn_val = prev_close - dif_max
    return rtn_val
        
def doPredict1(next_date_range, model1, etf_dic, next_pd_dic, scaler_dic, feature_cols, etf_ids):
    predict_res_final_1 = []
    dummy_date = '20999999'
    for date in next_date_range:
        for eid in etf_ids:
            #do predict
            pred_data = next_pd_dic[eid.strip()][feature_cols].values
            pred_res = model1.predict(data=pred_data)
#             print(pred_res)

            etf_dt = etf_dic[eid.strip()]
            prev_dt = etf_dt.loc[(etf_dt["etf_date"] < date) ].iloc[-1]
#             print('id of prev_dt: ', eid, ', date: ' ,prev_dt["etf_date"])
            prev_close = prev_dt["etf_close"]
            scaler = scaler_dic['close_dif']
            prediction = prev_close + scaler.inverse_transform(pred_res[0])[0][0]
            prediction = max_7percent(prev_close, prediction)
            #取出所有預測結果作合併，以進行後續成績計算
            predict_res_final_1.append({
                'etf_id': eid, 'etf_date': date,
                'prediction': prediction, 'pred_ud': judge_up_down_pred_native(prediction, prev_close)
            })

            if (date != next_date_range[-1]) and (date != dummy_date):
                #作出新的next_pd
                new_dic = {'etf_id': [eid], 'etf_date': [date], 'etf_name': [''], 
                           'etf_open': [0.0],  'etf_high': [0.0], 'etf_low': [0.0], 'etf_close': [prediction], 'etf_count': [0.0]}
                new_pd = pd.DataFrame(data=new_dic)[['etf_id','etf_date','etf_name', 'etf_open', 'etf_high', 
                                                          'etf_low', 'etf_close', 'etf_count']]
                #print(new_pd)
                etf_dt=etf_dt.append(new_pd, ignore_index=True)
                next_date = next_date_range[next_date_range.index(date)+1]
                etf_pd2, next_pd2, _ = create_feature(etf_dt, next_date, scaler_dic)
                etf_dic.update({eid.strip(): etf_dt})
                next_pd_dic.update({eid.strip(): next_pd2})
    return predict_res_final_1
    
def doPredict2(next_date_range, model2, etf_dic, next_pd_dic, scaler_dic, feature_cols, etf_ids):
    predict_res_final_2 = []
    for date in next_date_range:
        for eid in etf_ids:
            #do predict
            pred_data = next_pd_dic[eid.strip()][feature_cols].values
            pred_res = model2.predict(data=pred_data)
#             print(eid.strip(), '-', date, ': ', pred_res[0], '-', 
#                   scaler_dic[eid.strip()+'-etf_close'].inverse_transform(pred_res[0])[0][0])

            etf_dt = etf_dic[eid.strip()]
            prev_dt = etf_dt.loc[(etf_dt["etf_date"] < date) ].iloc[-1]
    #         print('id of prev_dt: ', eid, ', date: ' ,prev_dt["etf_date"])
            prev_close = prev_dt["etf_close"]
            scaler = scaler_dic['etf_close']
            prediction = scaler.inverse_transform(pred_res[0])[0][0]
            prediction = max_7percent(prev_close, prediction)
            #取出所有預測結果作合併，以進行後續成績計算
            predict_res_final_2.append({
                'etf_id': eid, 'etf_date': date,
                'prediction': prediction, 'pred_ud': judge_up_down_pred_native(prediction, prev_close)
            })

            if date != next_date_range[-1]:
                #作出新的next_pd
                new_dic = {'etf_id': [eid], 'etf_date': [date], 'etf_name': [''], 
                           'etf_open': [0.0],  'etf_high': [0.0], 'etf_low': [0.0], 'etf_close': [prediction], 'etf_count': [0.0]}
                new_pd = pd.DataFrame(data=new_dic)[['etf_id','etf_date','etf_name', 'etf_open', 'etf_high', 
                                                          'etf_low', 'etf_close', 'etf_count']]
                #print(new_pd)
                etf_dt=etf_dt.append(new_pd, ignore_index=True)
                next_date = next_date_range[next_date_range.index(date)+1]
                etf_pd2, next_pd2, _ = create_feature(etf_dt, next_date, scaler_dic)
                etf_dic.update({eid.strip(): etf_dt})
                next_pd_dic.update({eid.strip(): next_pd2})
    return predict_res_final_2

In [27]:
# for subset in itertools.combinations(feature_cols, 1):
#     print(list(subset))

In [28]:
#$$$
feature_date_dic = {
    'ema5_slope,ema30,macd1,bias_ma':'2-20180604',
    'ppo,ema30,d,ema3_slope':'2-20180604',
    'ppo,ema30,d,ema20':'2-20180604',
    'macd1,ema30,ema5_slope,macdsignal1':'2-20180604',
    'ema5_slope,ema30,macd1,ppo':'2-20180604',
    'ema5_slope,ema30,macd1,dif':'2-20180604',
    'ema30,ppo,ema10_slope,bias_3_6':'2-20180604',
    'ppo,ema20,nbias6,ema5_slope':'2-20180604',
    'ppo,ema20,ema3_slope,macdsignal1':'2-20180604',
    'ema30,ppo,ema5_slope,nbias3':'2-20180604',
    'ppo,ema20,ema3_slope,ema30':'2-20180604',
    'ppo,ema30,ema10_slope,bias_ma':'2-20180604',
    'ema30,ppo,nbias6,ema20':'2-20180604',
    'ppo,ema30,nbias6,ema20':'2-20180604',
    'ema30,ppo,ema5_slope,dif':'2-20180604',



    'ema20,rocr,ema3_slope,ema5_slope':'2-20180528',
    'ema20,rocr,nbias3,ema3_slope':'2-20180528',
    'ema20,nbias6,rocr,ema5_slope':'2-20180528',
    'ema20,rocr,ema10,cmo':'2-20180528',
    'nbias3,ema20,ema30,macdsignal1':'2-20180528',
    'ema20,rocr,nbias3,ppo':'2-20180528',
    'ema20,rocr,nbias3,macdsignal1':'2-20180528',
    'ema20,d,ema30,bias_ma':'1-20180528',
    'ema5,k,macdsignal1,nbias3':'2-20180528',
    'ema20,rocr,ema3_slope,ppo':'2-20180528',
    'ema20,rocr,ema3_slope,nbias3':'2-20180528',
    'ema20,rocr,d,ema3_slope':'2-20180528',
    'nbias6,ema3_slope,macdsignal1,ema30':'1-20180528',
    'ema20,rocr,ema3_slope,nbias6':'2-20180528',
    'nbias3,ema20,ema30,rocr':'2-20180528',



    'ema3,rocr,nbias20,dif':'2-20180521',
    'ema10,rocr,macdhist1,nbias20':'1-20180521',
    'ema5,dif,nbias20,ema20':'2-20180521',
    'ema10,rocr,d,nbias20':'2-20180521',
    'ema10,rocr,nbias20,ema10_slope':'1-20180521',
    'ema10,rocr,macdhist1,d':'1-20180521',
    'ema3,rocr,nbias3,ema20':'1-20180521',
    'ema10,rocr,dif,nbias20':'2-20180521',
    'ema5,dif,nbias20,rocr':'2-20180521',
    'ema30,ppo,rocr,macdsignal1':'2-20180521',
    'ema3,rsi,d,ema20':'2-20180521',
    'ema5,nbias20,rocr,cmo':'1-20180521',
    'ema3,rsi,dif,ema20':'2-20180521',
    'ema5,dif,nbias20,macd1':'2-20180521',
    'ema3,rocr,nbias20,macdhist1':'1-20180521',



    'ema3_slope,ema20,dif,ema5_slope':'2-20180514',
    'ema3_slope,ema10,d,ema30':'2-20180514',
    'ema3_slope,ema20,dif,d':'2-20180514',
    'ema3_slope,ema10,d,ppo':'2-20180514',
    'ema3_slope,ema20,d,ema10':'2-20180514',
    'rocr,nbias3,ema3,ppo':'2-20180514',
    'ema3_slope,ema10,ema30,d':'2-20180514',
    'ema3_slope,ema10,ppo,bias_3_6':'2-20180514',
    'ema3_slope,ema10,ppo,nbias3':'2-20180514',
    'ema3_slope,ema10,ppo,ema30':'2-20180514',
    'ema3_slope,ema10,ppo,k':'2-20180514',
    'ema3_slope,ema10,ema30,bias_ma':'2-20180514',
    'ema3_slope,ema20,dif,bias_3_6':'2-20180514',
    'k,ema30,ema5_slope,ema20':'2-20180514',
    'k,ema30,ema5_slope,macdsignal1':'2-20180514',



    'ema20,ema10_slope,nbias3,bias_3_6':'2-20180507',
    'ema30,ema10_slope,ema3_slope,k':'2-20180507',
    'ema20,ema5_slope,nbias3,bias_3_6':'2-20180507',
    'ema3_slope,ema10_slope,ema20,nbias3':'2-20180507',
    'ema10_slope,ema20,nbias3,ema3_slope':'2-20180507',
    'ema20,ema10_slope,ema3_slope,nbias3':'2-20180507',
    'ema10_slope,ema20,nbias3,rocr':'2-20180507',
    'ema10_slope,ema20,ema3_slope,bias_3_6':'2-20180507',
    'ema10_slope,ema20,nbias3,ema5_slope':'2-20180507',
    'ema10_slope,ema20,nbias3,nbias6':'2-20180507',
    'ema20,ema10_slope,nbias3,k':'2-20180507',
    'ema10,ema5_slope,nbias3,ema20':'2-20180507',
    'ema3_slope,ema10_slope,ema30,nbias3':'2-20180507',
    'ema3_slope,ema10_slope,ema20,nbias6':'2-20180507',
    'ema20,ema10_slope,nbias3,bias_ma':'2-20180507',



    'bias_3_6,ema30,bias_ma,nbias6':'2-20180611',
    'bias_3_6,ema30,ema3_slope,ema5_slope':'2-20180611',
    'ema30,bias_3_6,nbias3,d':'2-20180611',
    'bias_3_6,ema30,nbias6,bias_ma':'2-20180611',
    'ema20,nbias6,ppo,bias_3_6':'2-20180611',
    'ema20,nbias6,ppo,macdhist1':'2-20180611',
    'bias_3_6,ema30,ema3_slope,nbias6':'2-20180611',
    'ema20,nbias6,nbias3,bias_3_6':'2-20180611',
    'bias_3_6,ema30,ema3_slope,nbias3':'2-20180611',
    'ema20,nbias6,nbias3,ema30':'2-20180611',
    'ema30,bias_3_6,bias_ma,ema3_slope':'2-20180611',
    'bias_3_6,ema30,bias_ma,ema5_slope':'2-20180611',
    'rsi,macd1,d,ema30':'1-20180611',
    'ema30,bias_3_6,nbias3,macdsignal1':'2-20180611',
    'ema30,bias_3_6,ema5_slope,bias_ma':'2-20180611'
}
feature_dic = {}
f_date_dic = {}
for feature_str in feature_date_dic.keys():
    type_date = feature_date_dic[feature_str]
    type_date_seg = type_date.split('-')
    f_type = type_date_seg[0]
    f_date = type_date_seg[1]
    feature_dic.update({feature_str: f_type})
    f_date_dic.update({feature_str: f_date})

In [29]:
def train_and_eval(feature_dic, f_date_dic, etf_dic, train_dt, next_pd_dic, date_range, scaler_dic, etf_ids, tetf_pd_eval):    
    type_list = []
    feature_list = []
    acc_list = []
    rmse_list = []
    escore_list = []
    all_pred_list = []
    etf_pd_dic = {}
    predict_start_date = date_range[0]
    # for features in itertools.combinations(feature_cols, feature_num):
    for feature_str in feature_dic.keys():    
        f_type = feature_dic[feature_str]
        f_date = f_date_dic[feature_str]
        if f_date == predict_start_date:
            continue
        feature_cols = list(feature_str.split(','))
        model = trainModel(train_dt, feature_cols, f_type)
        #recover to re-run
        for eid in etf_ids:
            etf_dt = etf_dic[eid.strip()]
            etf_dt = etf_dt.loc[~etf_dt["etf_date"].isin(date_range)]
            etf_dic.update({eid.strip(): etf_dt})
            etf_pd, next_pd, _ = create_feature(etf_dic[eid.strip()], predict_start_date, scaler_dic)
    #         print(eid.strip(), ': ', etf_pd.shape)
            etf_pd_dic.update({eid.strip() : etf_pd})
            next_pd_dic.update({eid.strip() : next_pd})
        if f_type == '1':
            #evaluate close_dif
            predict_res_pd_1 = doPredict1(date_range, model, etf_dic, next_pd_dic, scaler_dic, feature_cols, etf_ids)
            all_pred_list = all_pred_list + predict_res_pd_1

            rmse1, acc1, e_score1 = doEvaluate(predict_res_pd_1, tetf_pd_eval, date_range)
            type_list.append("1")
            feature_list.append(feature_str)
            acc_list.append(acc1)
            rmse_list.append(rmse1)
            escore_list.append(e_score1)
            print(feature_str, ' f_date: ', f_date, ' pred_date: ', predict_start_date,' final1=', e_score1)
        else:
            #evaluate etf_close
            predict_res_pd_2 = doPredict2(date_range, model, etf_dic, next_pd_dic, scaler_dic, feature_cols, etf_ids)
            all_pred_list = all_pred_list + predict_res_pd_2

            rmse2, acc2, e_score2 = doEvaluate(predict_res_pd_2, tetf_pd_eval, date_range)
            type_list.append("2")
            feature_list.append(feature_str)
            acc_list.append(acc2)
            rmse_list.append(rmse2)
            escore_list.append(e_score2)
            print(feature_str, ' f_date: ', f_date, ' pred_date: ', predict_start_date,' final2=', e_score2)

    score_dic = {
        'feature_type' : type_list, 'feature': feature_list, 'accuracy': acc_list, 'rmse': rmse_list,
        'final_score': escore_list, 'pred_date': predict_start_date
    }
    
    return score_dic

def create_train_dt(train_etf_dic, date_range,etf_ids):
    etf_pd_dic = {}
    next_pd_dic = {}
    scaler_dic = {} #save minmax scaler
    predict_start_date = date_range[0]
    for etfid in etf_ids:
        etf_pd, next_pd, scalers = create_feature(train_etf_dic[etfid.strip()], predict_start_date, {})
        etf_pd_dic.update({etfid.strip() : etf_pd})
        next_pd_dic.update({etfid.strip() : next_pd})
        scaler_dic.update(scalers)
    train_dt_org = pd.concat([etf_pd_dic[etfid.strip()] for etfid in etf_ids])
    
    std_dic = {} # save reult of scaler
    for col in [c for c in train_dt_org.columns.values if c not in ["etf_id", "etf_date"]]:
            scaler_key = col
            if scaler_key in scaler_dic:
                scaler = scaler_dic[scaler_key]
                merged = list(itertools.chain.from_iterable(scaler.transform(train_dt_org[[col]])))                
            else:
                scaler = MinMaxScaler()
                merged = list(itertools.chain.from_iterable(scaler.fit_transform(train_dt_org[[col]])))
                #保留etf_close的scaler在後續使用
                scaler_dic.update({scaler_key: scaler})
            std_dic.update({col: merged})
    train_dt = pd.DataFrame(std_dic)
    print('train_dt count:', train_dt.shape)
    
    for etfid in etf_ids:
        std_dic = {}
        next_pd = next_pd_dic[etfid.strip()]
        for col in [c for c in next_pd.columns.values if c not in ["etf_id", "etf_date"]]:
            scaler = scaler_dic[col]
            merged = list(itertools.chain.from_iterable(scaler.transform(next_pd[[col]])))
            std_dic.update({col: merged})
        next_pd_dic.update({etfid.strip() : pd.DataFrame(std_dic)})
    print('next_pd_dic count:', len(next_pd_dic.keys()), '*' , next_pd_dic['0050'].shape)
    return (train_dt, next_pd_dic, scaler_dic)

In [30]:
score_pd = None
for date_range in train_date_range:
    train_etf_dic = {}
    for eid in etf_ids:
        end_train_date = date_range[0]
        etf_dt = etf_dic[eid.strip()]
        train_etf_dt = etf_dt.loc[(etf_dt['etf_id'] == eid) & (etf_dt['etf_date'] < end_train_date)]
        train_etf_dic.update({eid.strip(): train_etf_dt})
    train_dt, next_pd_dic, scaler_dic = create_train_dt(train_etf_dic, date_range, etf_ids)
    score_dic = train_and_eval(feature_dic, f_date_dic, train_etf_dic, train_dt, next_pd_dic, 
                                date_range, scaler_dic, etf_ids, tetf_pd_eval)
    if score_pd is None:
        score_pd = pd.DataFrame(data=score_dic)[['feature_type', 'feature', 'pred_date', 'accuracy', 'rmse', 'final_score']]
    else:
        tmp_pd = pd.DataFrame(data=score_dic)[['feature_type', 'feature', 'pred_date', 'accuracy', 'rmse', 'final_score']]
        score_pd = pd.concat([score_pd, tmp_pd])

train_dt count: (15531, 26)
next_pd_dic count: 18 * (1, 26)
ema5_slope,ema30,macd1,bias_ma  f_date:  20180604  pred_date:  20180507  final2= 14.586252967251632
ppo,ema30,d,ema3_slope  f_date:  20180604  pred_date:  20180507  final2= 13.818422229474814
ppo,ema30,d,ema20  f_date:  20180604  pred_date:  20180507  final2= 13.529442219910473
macd1,ema30,ema5_slope,macdsignal1  f_date:  20180604  pred_date:  20180507  final2= 14.284990692985238
ema5_slope,ema30,macd1,ppo  f_date:  20180604  pred_date:  20180507  final2= 14.403337473607682
ema5_slope,ema30,macd1,dif  f_date:  20180604  pred_date:  20180507  final2= 14.022995987292669
ema30,ppo,ema10_slope,bias_3_6  f_date:  20180604  pred_date:  20180507  final2= 14.986337558687381
ppo,ema20,nbias6,ema5_slope  f_date:  20180604  pred_date:  20180507  final2= 13.119584792418667
ppo,ema20,ema3_slope,macdsignal1  f_date:  20180604  pred_date:  20180507  final2= 12.782939873012445
ema30,ppo,ema5_slope,nbias3  f_date:  20180604  pred_date:  201805

ppo,ema20,ema3_slope,ema30  f_date:  20180604  pred_date:  20180514  final2= 13.901971190131075
ppo,ema30,ema10_slope,bias_ma  f_date:  20180604  pred_date:  20180514  final2= 13.54092101526224
ema30,ppo,nbias6,ema20  f_date:  20180604  pred_date:  20180514  final2= 14.53841698807588
ppo,ema30,nbias6,ema20  f_date:  20180604  pred_date:  20180514  final2= 14.19291331505131
ema30,ppo,ema5_slope,dif  f_date:  20180604  pred_date:  20180514  final2= 15.114453269741011
ema20,rocr,ema3_slope,ema5_slope  f_date:  20180528  pred_date:  20180514  final2= 12.865857969741894
ema20,rocr,nbias3,ema3_slope  f_date:  20180528  pred_date:  20180514  final2= 13.232005708894716
ema20,nbias6,rocr,ema5_slope  f_date:  20180528  pred_date:  20180514  final2= 12.30678966135571
ema20,rocr,ema10,cmo  f_date:  20180528  pred_date:  20180514  final2= 13.321636664653523
nbias3,ema20,ema30,macdsignal1  f_date:  20180528  pred_date:  20180514  final2= 12.764446120149206
ema20,rocr,nbias3,ppo  f_date:  20180528  p

nbias3,ema20,ema30,macdsignal1  f_date:  20180528  pred_date:  20180521  final2= 12.670416692568777
ema20,rocr,nbias3,ppo  f_date:  20180528  pred_date:  20180521  final2= 13.292643682792178
ema20,rocr,nbias3,macdsignal1  f_date:  20180528  pred_date:  20180521  final2= 12.612467449834549
ema20,d,ema30,bias_ma  f_date:  20180528  pred_date:  20180521  final1= 12.383502069907287
ema5,k,macdsignal1,nbias3  f_date:  20180528  pred_date:  20180521  final2= 13.283958444018664
ema20,rocr,ema3_slope,ppo  f_date:  20180528  pred_date:  20180521  final2= 14.026726990258911
ema20,rocr,ema3_slope,nbias3  f_date:  20180528  pred_date:  20180521  final2= 12.992687386078037
ema20,rocr,d,ema3_slope  f_date:  20180528  pred_date:  20180521  final2= 12.93901638367309
nbias6,ema3_slope,macdsignal1,ema30  f_date:  20180528  pred_date:  20180521  final1= 12.659435801564937
ema20,rocr,ema3_slope,nbias6  f_date:  20180528  pred_date:  20180521  final2= 13.429302554447021
nbias3,ema20,ema30,rocr  f_date:  20

ema5,dif,nbias20,macd1  f_date:  20180521  pred_date:  20180528  final2= 13.292677520812454
ema3,rocr,nbias20,macdhist1  f_date:  20180521  pred_date:  20180528  final1= 13.87489659410947
ema3_slope,ema20,dif,ema5_slope  f_date:  20180514  pred_date:  20180528  final2= 12.867958160963111
ema3_slope,ema10,d,ema30  f_date:  20180514  pred_date:  20180528  final2= 14.199166683083353
ema3_slope,ema20,dif,d  f_date:  20180514  pred_date:  20180528  final2= 14.381314428589938
ema3_slope,ema10,d,ppo  f_date:  20180514  pred_date:  20180528  final2= 13.06973452215816
ema3_slope,ema20,d,ema10  f_date:  20180514  pred_date:  20180528  final2= 13.681101752799393
rocr,nbias3,ema3,ppo  f_date:  20180514  pred_date:  20180528  final2= 13.611005232297769
ema3_slope,ema10,ema30,d  f_date:  20180514  pred_date:  20180528  final2= 13.63813095121478
ema3_slope,ema10,ppo,bias_3_6  f_date:  20180514  pred_date:  20180528  final2= 13.164528331541154
ema3_slope,ema10,ppo,nbias3  f_date:  20180514  pred_date:

ema3_slope,ema10,ppo,nbias3  f_date:  20180514  pred_date:  20180604  final2= 14.506772392116929
ema3_slope,ema10,ppo,ema30  f_date:  20180514  pred_date:  20180604  final2= 14.545610191094699
ema3_slope,ema10,ppo,k  f_date:  20180514  pred_date:  20180604  final2= 14.423676454848447
ema3_slope,ema10,ema30,bias_ma  f_date:  20180514  pred_date:  20180604  final2= 13.525780353211141
ema3_slope,ema20,dif,bias_3_6  f_date:  20180514  pred_date:  20180604  final2= 14.073119709066175
k,ema30,ema5_slope,ema20  f_date:  20180514  pred_date:  20180604  final2= 13.678504201997049
k,ema30,ema5_slope,macdsignal1  f_date:  20180514  pred_date:  20180604  final2= 13.740242091313114
ema20,ema10_slope,nbias3,bias_3_6  f_date:  20180507  pred_date:  20180604  final2= 11.906425022768362
ema30,ema10_slope,ema3_slope,k  f_date:  20180507  pred_date:  20180604  final2= 13.000187194896819
ema20,ema5_slope,nbias3,bias_3_6  f_date:  20180507  pred_date:  20180604  final2= 12.838334255516486
ema3_slope,ema10_

ema3_slope,ema20,dif,d  f_date:  20180514  pred_date:  20180611  final2= 13.248990637323102
ema3_slope,ema10,d,ppo  f_date:  20180514  pred_date:  20180611  final2= 13.37930238278601
ema3_slope,ema20,d,ema10  f_date:  20180514  pred_date:  20180611  final2= 13.179332143245126
rocr,nbias3,ema3,ppo  f_date:  20180514  pred_date:  20180611  final2= 12.980035069406505
ema3_slope,ema10,ema30,d  f_date:  20180514  pred_date:  20180611  final2= 12.678658070808122
ema3_slope,ema10,ppo,bias_3_6  f_date:  20180514  pred_date:  20180611  final2= 13.62799685997301
ema3_slope,ema10,ppo,nbias3  f_date:  20180514  pred_date:  20180611  final2= 13.826878601324886
ema3_slope,ema10,ppo,ema30  f_date:  20180514  pred_date:  20180611  final2= 13.082585207021472
ema3_slope,ema10,ppo,k  f_date:  20180514  pred_date:  20180611  final2= 12.88889577236037
ema3_slope,ema10,ema30,bias_ma  f_date:  20180514  pred_date:  20180611  final2= 13.464876154598612
ema3_slope,ema20,dif,bias_3_6  f_date:  20180514  pred_da

In [31]:
# etf_dic['0050']
# score_dic['feature']

In [32]:
# find top 7 feature combinations
f_str_list = []
f_score_list = []
f_type_list = []
for feature_str in feature_dic.keys():
    f_pd = score_pd.loc[score_pd.feature == feature_str]
    f_scores = f_pd['final_score'].values.tolist()
    if f_scores is None or len(f_scores) == 0:
        continue
    f_score_avg = avg_list(f_scores)
    f_str = f_pd['feature'].values[0]
    f_type_list.append(f_pd['feature_type'].values[0])
    f_str_list.append(f_str)
    f_score_list.append(f_score_avg)

feature_filter_pd_tmp = pd.DataFrame({"feature": f_str_list, "final_score": f_score_list, "f_type" : f_type_list})
feature_filter_pd_tmp.shape
len(f_score_list)

90

In [33]:
feature_filter_pd=feature_filter_pd_tmp.loc[feature_filter_pd_tmp.final_score > 13.55] \
                .sort_values(by=['final_score'], ascending=[0])
feature_filter_pd.shape

(30, 3)

In [34]:
def train_and_eval_final(feature_dic, etf_dic, train_dt, next_pd_dic, date_range, scaler_dic, etf_ids, tetf_pd_eval, doEval):    
    type_list = []
    feature_list = []
    acc_list = []
    rmse_list = []
    escore_list = []
    all_pred_list = []
    etf_pd_dic = {}
    predict_start_date = date_range[0]
    # for features in itertools.combinations(feature_cols, feature_num):
    for feature_str in feature_dic.keys():    
        f_type = feature_dic[feature_str]
        feature_cols = list(feature_str.split(','))
        model = trainModel(train_dt, feature_cols, f_type)
        #recover to re-run
        for eid in etf_ids:
            etf_dt = etf_dic[eid.strip()]
            etf_dt = etf_dt.loc[~etf_dt["etf_date"].isin(date_range)]
            etf_dic.update({eid.strip(): etf_dt})
            etf_pd, next_pd, _ = create_feature(etf_dic[eid.strip()], predict_start_date, scaler_dic)
    #         print(eid.strip(), ': ', etf_pd.shape)
            etf_pd_dic.update({eid.strip() : etf_pd})
            next_pd_dic.update({eid.strip() : next_pd})
        if f_type == '1':
            #evaluate close_dif
            predict_res_pd_1 = doPredict1(date_range, model, etf_dic, next_pd_dic, scaler_dic, feature_cols, etf_ids)
            all_pred_list = all_pred_list + predict_res_pd_1
            if doEval:
                rmse1, acc1, e_score1 = doEvaluate(predict_res_pd_1, tetf_pd_eval, date_range)
                type_list.append("1")
                feature_list.append(feature_str)
                acc_list.append(acc1)
                rmse_list.append(rmse1)
                escore_list.append(e_score1)
                print(feature_str, ' f_date: ', f_date, ' pred_date: ', predict_start_date,' final1=', e_score1)
        else:
            #evaluate etf_close
            predict_res_pd_2 = doPredict2(date_range, model, etf_dic, next_pd_dic, scaler_dic, feature_cols, etf_ids)
            all_pred_list = all_pred_list + predict_res_pd_2
            if doEval:
                rmse2, acc2, e_score2 = doEvaluate(predict_res_pd_2, tetf_pd_eval, date_range)
                type_list.append("2")
                feature_list.append(feature_str)
                acc_list.append(acc2)
                rmse_list.append(rmse2)
                escore_list.append(e_score2)
                print(feature_str, ' f_date: ', f_date, ' pred_date: ', predict_start_date,' final2=', e_score2)

    score_dic = {
        'feature_type' : type_list, 'feature': feature_list, 'accuracy': acc_list, 'rmse': rmse_list,
        'final_score': escore_list, 'pred_date': predict_start_date
    }
    
    return (score_dic, all_pred_list)

In [35]:
doEval = False
score_pd = None
final_etf_dic = {}

final_feature_dic = {}
for index, row in feature_filter_pd.iterrows():
    f_str = row['feature']
    f_type = row['f_type']
    final_feature_dic.update({f_str: f_type})

for eid in etf_ids:
    end_date = pred_date_range[0]
    etf_dt = etf_dic[eid.strip()]
    final_etf_dt = etf_dt.loc[(etf_dt['etf_id'] == eid) & (etf_dt['etf_date'] < end_date)]
    final_etf_dic.update({eid.strip(): final_etf_dt})
train_dt, next_pd_dic, scaler_dic = create_train_dt(final_etf_dic, pred_date_range, etf_ids)
score_dic,all_pred_list = train_and_eval_final(final_feature_dic, final_etf_dic, train_dt, next_pd_dic, 
                                               pred_date_range, scaler_dic, etf_ids, tetf_pd_eval, doEval)
if score_pd is None:
    score_pd = pd.DataFrame(data=score_dic)[['feature_type', 'feature', 'pred_date', 'accuracy', 'rmse', 'final_score']]
else:
    tmp_pd = pd.DataFrame(data=score_dic)[['feature_type', 'feature', 'pred_date', 'accuracy', 'rmse', 'final_score']]
    score_pd = pd.concat([score_pd, tmp_pd])

train_dt count: (16071, 26)
next_pd_dic count: 18 * (1, 26)


In [36]:
# eid = '0050   '
# end_date = pred_date_range[0]
# etf_dt = etf_dic[eid.strip()]
# print(etf_dt.shape)
# final_etf_dt = etf_dt.loc[(etf_dt['etf_id'] == eid) & (etf_dt['etf_date'] < end_date)]
# final_etf_dt

In [37]:
def judge_ud(ud_list):
    num_1 = 0
    num_0 = 0
    num_2 = 0
    for ud in ud_list:
        if ud == 1.0:
            num_1 += 1
        elif ud == 2.0:
            num_2 += 1
        else:
            num_0 += 1
    if num_2 >= num_1 and num_2 >= num_0:
        return 2.0
    if num_1 >= num_2 and num_1 >= num_0:
        return 1.0
    return 0.0

all_pred_pd = pd.DataFrame(all_pred_list)
predict_res_final = []
for date in pred_date_range:
    for eid in etf_ids:
        pred_pd = all_pred_pd.loc[(all_pred_pd['etf_id'] == eid) & (all_pred_pd['etf_date'] == date)]
        ud_list = pred_pd['pred_ud'].tolist()
        ud = judge_ud(ud_list)
        close_list = pred_pd["prediction"].tolist()
        close = avg_list(close_list)
        predict_res_final.append({
            'etf_id': eid, 'etf_date': date,
            'prediction': close, 'pred_ud': ud
        })

if doEval:
    rmse, acc, e_score = doEvaluate(predict_res_final, tetf_pd_eval, pred_date_range)
    print(predict_start_date, ' final: rmse=', rmse, ', accuracy=', acc, 'etf_score=', e_score)

# 20180611  final: rmse= 0.15960512867137927 , accuracy= 0.6111111111111112 etf_score= 14.30800929702666 (feature score thresh > 13.55)
# 20180604  final: rmse= 0.42848136168489154 , accuracy= 0.7222222222222222 etf_score= 15.078408223483088
# 20180528  final: rmse= 0.3379072596968221 , accuracy= 0.5222222222222223 etf_score= 13.634738477828757
# 20180521  final: rmse= 0.3481860950449327 , accuracy= 0.5333333333333333 etf_score= 14.153413219827922
#$$$

In [38]:
final_res = spark.createDataFrame(pd.DataFrame(predict_res_final)).orderBy("etf_id", "etf_date").collect()
#export to pandas
etf_ids = []
mon_ud = []
mon_price = [] 
tue_ud = []
tue_price = []
wed_ud = []
wed_price = []
thu_ud = []
thu_price = []
fri_ud = []
fri_price = []

def encode_ud(oper_ud):
    if oper_ud == 0.0:
        return 0
    elif oper_ud == 1.0:
        return 1
    else:
        return -1

for row in final_res:
    etf_id = row["etf_id"]
    if etf_id not in etf_ids:
        etf_ids.append(etf_id)
    etf_date = row["etf_date"]
    eidx = pred_date_range.index(row["etf_date"])
    if eidx == 0:
        mon_ud.append(encode_ud(row["pred_ud"]))
        mon_price.append(row["prediction"])
    elif eidx == 1:
        tue_ud.append(encode_ud(row["pred_ud"]))
        tue_price.append(row["prediction"])
    elif eidx == 2:
        wed_ud.append(encode_ud(row["pred_ud"]))
        wed_price.append(row["prediction"])
    elif eidx == 3:
        thu_ud.append(encode_ud(row["pred_ud"]))
        thu_price.append(row["prediction"])
    elif eidx == 4:
        fri_ud.append(encode_ud(row["pred_ud"]))
        fri_price.append(row["prediction"])
        
if len(mon_ud) == 0:
    mon_ud = list(0 for i in range(0,len(etf_ids)))
    mon_price = list(0.0 for i in range(0,len(etf_ids)))
if len(tue_ud) == 0:
    tue_ud = list(0 for i in range(0,len(etf_ids)))
    tue_price = list(0.0 for i in range(0,len(etf_ids)))
if len(wed_ud) == 0:
    wed_ud = list(0 for i in range(0,len(etf_ids)))
    wed_price = list(0.0 for i in range(0,len(etf_ids)))
if len(thu_ud) == 0:
    thu_ud = list(0 for i in range(0,len(etf_ids)))
    thu_price = list(0.0 for i in range(0,len(etf_ids)))
if len(fri_ud) == 0:
    fri_ud = list(0 for i in range(0,len(etf_ids)))
    fri_price = list(0.0 for i in range(0,len(etf_ids)))
    
dic = {"ETFid": etf_ids, 
       "Mon_ud": mon_ud, "Mon_cprice": mon_price,
       "Tue_ud": tue_ud, "Tue_cprice": tue_price,
       "Wed_ud": wed_ud, "Wed_cprice": wed_price,
       "Thu_ud": thu_ud, "Thu_cprice": thu_price,
       "Fri_ud": fri_ud, "Fri_cprice": fri_price
      }
final_df = pd.DataFrame(data=dic)[['ETFid','Mon_ud','Mon_cprice','Tue_ud','Tue_cprice',
                                  'Wed_ud','Wed_cprice','Thu_ud','Thu_cprice',
                                  'Fri_ud','Fri_cprice']]
final_df

,ETFid,Mon_ud,Mon_cprice,Tue_ud,Tue_cprice,Wed_ud,Wed_cprice,Thu_ud,Thu_cprice,Fri_ud,Fri_cprice
0,0050,1,82.226012,1,82.353621,1,82.481377,1,82.487849,0,0.0
1,0051,1,34.481686,1,34.540491,1,34.704316,1,34.915979,0,0.0
2,0052,1,51.803076,1,52.112956,1,52.457362,1,52.820472,0,0.0
3,0053,1,36.471029,1,36.531018,1,36.567099,1,36.753537,0,0.0
4,0054,-1,24.532665,1,24.541062,1,24.667694,1,24.747576,0,0.0
5,0055,1,17.690635,1,17.779023,1,17.821215,-1,17.813587,0,0.0
6,0056,-1,26.289878,-1,26.208391,-1,26.167290,1,26.192496,0,0.0
7,0057,1,50.326776,1,50.426008,1,50.516466,1,50.659876,0,0.0
8,0058,1,47.695546,1,47.759942,1,47.897349,1,47.939630,0,0.0
9,0059,1,42.927015,1,43.030200,1,43.054301,1,43.160905,0,0.0


In [39]:
final_df.to_csv(Path.replace("file:","") + "/etf_price_pred_talib_eval_mac_4f_" + pred_date_range[0] +".csv",index=False)